In [1]:
LPNE_PATH = "./lpne-data-analysis/"
OLD_MICE_PATH = "/work/mk423/Clock-D19-EPM/Clock_d19_saveFeatures_1_2.mat"
NEW_MICE_PATH = "/work/mk423/Clock-D19-EPM/Clock-D19-New-Mice-SaveFeatures-1-2.mat"
RUN_SHEET = "/work/mk423/Clock-D19-EPM/EPM_CLOCK Spreadsheet 2021.xlsx"
SAVE_FILE = "/work/mk423/Clock-D19-EPM/C19_EPM_Data.pkl"
FEATURE_LIST = ["power","coherence","granger"]
import os, sys
sys.path.append(LPNE_PATH)
import numpy as np
import pandas as pd
import pickle
import data_tools

In [2]:
run_df = pd.read_excel(RUN_SHEET)
run_df.head()

,MOUSE,EXPDATE,SEX,CONDITION,START TIME
0,Mouse0911,22021,F,WT,216
1,Mouse0912,22021,F,CLOCK,205
2,Mouse0913,22021,F,CLOCK,200
3,Mouse0914,22021,F,WT,199
4,Mouse2401,30321,M,CLOCK,336


In [3]:
X_psd_old,X_coh_old,X_gc_old,labels_old= data_tools.load_data(OLD_MICE_PATH,feature_list=FEATURE_LIST)
X_psd_new,X_coh_new,X_gc_new,labels_new = data_tools.load_data(NEW_MICE_PATH,feature_list=FEATURE_LIST)

version saveFeatures_1.2 used to calcuate power features
version saveFeatures_1.2 used to calcuate coherence features


./lpne-data-analysis/data_tools.py:144: UserWarning: Raw Granger causality values loaded; if you want exponentiated or capped features, you must do that yourself
  warnings.warn("Raw Granger causality values loaded; if you want exponentiated or capped features, you must do that yourself")


version saveFeatures_1.2 used to calcuate granger features
Version preprocessData_1.0 of preprocessing used
Make sure feature versions listed above match those used for any other dataset in the same project
version saveFeatures_1.2 used to calcuate power features
version saveFeatures_1.2 used to calcuate coherence features
version saveFeatures_1.2 used to calcuate granger features
Version preprocessData_1.0 of preprocessing used
Make sure feature versions listed above match those used for any other dataset in the same project


In [10]:
X_psd = np.vstack((X_psd_old,X_psd_new))
X_coh = np.vstack((X_coh_old,X_coh_new))
X_gc = np.vstack((X_gc_old,X_gc_new))


y_time_aw = np.hstack((labels_old['allWindows']['time'],labels_new['allWindows']['time']))
y_mouse_aw = np.hstack((labels_old['allWindows']['mouse'],labels_new['allWindows']['mouse']))
y_expDate_aw = np.hstack((labels_old['allWindows']['expDate'],labels_new['allWindows']['expDate']))

y_time = np.hstack((labels_old['windows']['time'],labels_new['windows']['time']))
y_mouse = np.hstack((labels_old['windows']['mouse'],labels_new['windows']['mouse']))
y_expDate = np.hstack((labels_old['windows']['expDate'],labels_new['windows']['expDate']))

In [5]:
y_task = np.zeros(y_time.shape)*np.nan
y_sex = []
y_condition = []

for mouse in y_mouse:
    y_sex.append(run_df[run_df["MOUSE"]==mouse]["SEX"].values[0])
    y_condition.append(run_df[run_df["MOUSE"]==mouse]["CONDITION"].values[0])
    
for mouse in np.unique(y_mouse):
    mouse_mask= y_mouse==mouse
    mouse_time = y_time[mouse_mask==1]
    y_task[mouse_mask==1] = mouse_time > run_df[run_df["MOUSE"]==mouse]["START TIME"].values[0]

### Make ROI tracking

In [19]:
ROI_FOLDER = "/work/mk423/Clock-D19-EPM/ROI_simba/"
y_roi = np.ones(y_time.shape)*np.nan
y_roi_aw = np.ones(y_time_aw.shape)*np.nan
files = os.listdir(ROI_FOLDER)
fps = 50
mouse_start_times = {}
for file in files:
    mouse = file.split("_")[0] #we only need mouse name since each mouse was only run on a single day

    start_time = run_df[run_df["MOUSE"]==mouse]["START TIME"].values[0]
    mouse_start_times[mouse] = start_time
        
    if mouse in np.unique(y_mouse):
        
        roi_df = pd.read_csv(ROI_FOLDER + file)
        
        mouse_mask = y_mouse == mouse
        mouse_mask_aw = y_mouse_aw == mouse
        
        start_time = run_df[run_df["MOUSE"]==mouse]["START TIME"].values[0]
        mouse_start_times[mouse] = start_time
        
        entry_frames = roi_df["Entry_frame"].to_numpy()
        exit_frames = roi_df["Exit_frame"].to_numpy()
        
        entry_times = entry_frames // fps #start time was accounted for in the video
        exit_times = exit_frames // fps  #so no offset is needed
        
        roi_seg = roi_df["Shape_name"].to_numpy()
        
        for roi, entry, exit in zip(roi_seg,entry_times,exit_times):
            time_mask = np.logical_and(y_time > entry, y_time <= exit)
            time_mask_aw = np.logical_and(y_time_aw > entry, y_time_aw <= exit)
            
            mask = np.logical_and(time_mask,mouse_mask)
            mask_aw = np.logical_and(time_mask_aw,mouse_mask_aw)
            
            y_roi[mask==1] = roi
            y_roi_aw[mask_aw==1] = roi


In [49]:
roi_df

,Animal_name,Shape_name,Entry_frame,Exit_frame
0,Animal_1_,3,17572,17573
1,Animal_1_,3,17738,17745
2,Animal_1_,3,17746,17752
3,Animal_1_,3,17756,17759
4,Animal_1_,3,17778,17783
...,...,...,...,...
355,Animal_1_,5,33127,33131
356,Animal_1_,5,33134,33152
357,Animal_1_,5,33155,33220
358,Animal_1_,5,33225,33240


### Make Pickle File

In [95]:
save_dict = {
    "X_psd":X_psd,
    "X_coh":X_coh,
    "X_gc":X_gc,
    "y_time":y_time,
    "y_mouse":y_mouse,
    "y_expDate":y_expDate,
    "y_sex":y_sex,
    "y_roi":y_roi,
    "y_condition":y_condition,
    "y_task":y_task,
    "labels_old_mice":labels_old,
    "labels_new_mice":labels_new,
    "feature version":"saveFeatures 1.2"
}

#with open(SAVE_FILE,"wb") as f:
#    pickle.dump(save_dict,f)

In [82]:
df = pd.DataFrame.from_dict({ "y_time":y_time,
    "y_mouse":y_mouse,
    "y_expDate":y_expDate,
    "y_sex":y_sex,
    "y_roi":y_roi,
    "y_condition":y_condition,
    "y_task":y_task})

### By Mouse Behavior Stats

In [43]:
mouse_list = []
counts_list = []
all_rois = np.array([1.,2.,3.,4.,5.,np.nan])
for mouse in np.unique(y_mouse_aw):
    mouse_list.append(mouse)
    mouse_mask = y_mouse_aw==mouse
    time_mask = np.logical_and(y_time_aw >= mouse_start_times[mouse],
                               y_time_aw < mouse_start_times[mouse] + 300)
    
    mask = np.logical_and(mouse_mask,time_mask)
    
    temp_rois, temp_counts = np.unique(y_roi_aw[mask==1],return_counts=True)
    
    temp_second_rois = [temp_counts[i-1] if i in temp_rois else 0 for i in range(1,6)]
    if np.isnan(temp_rois[-1]): 
        temp_second_rois.append(temp_counts[-1])
    else:
        temp_second_rois.append(0)
    
    counts_list.append(temp_second_rois)
    print(mouse, temp_second_rois)
    

Mouse0911 [234, 4, 7, 3, 40, 12]
Mouse0912 [76, 5, 134, 8, 67, 10]
Mouse0913 [42, 26, 84, 38, 48, 62]
Mouse0914 [10, 3, 215, 10, 27, 35]
Mouse2401 [41, 4, 37, 7, 92, 119]
Mouse2402 [71, 2, 123, 9, 74, 21]
Mouse2403 [54, 19, 60, 28, 94, 45]
Mouse30391 [162, 0, 4, 43, 42, 42]
Mouse30392 [137, 4, 83, 2, 42, 32]
Mouse3881 [242, 0, 5, 32, 11, 11]
Mouse3882 [48, 12, 92, 8, 129, 11]
Mouse3883 [56, 9, 46, 17, 147, 25]
Mouse69841 [168, 9, 30, 3, 75, 15]
Mouse69861 [249, 1, 15, 3, 19, 13]
Mouse69862 [153, 2, 91, 1, 30, 23]
Mouse69871 [0, 0, 0, 0, 0, 300]
Mouse69872 [112, 10, 24, 15, 97, 42]
Mouse69873 [112, 6, 93, 1, 68, 20]


In [38]:
roi_matrix = np.vstack(counts_list)
closed_arm_times = roi_matrix[:,0] + roi_matrix[:,2]
open_arm_times = roi_matrix[:,1] + roi_matrix[:,3]
center_times = roi_matrix[:,4]
nan_times = roi_matrix[:,5]

df_behavior = pd.DataFrame.from_dict({"mouse":mouse_list,
                                      "closed_arm_time":closed_arm_times,
                                      "open_arm_time":open_arm_times,
                                      "center_time":center_times,
                                      "nan_time":nan_times})

In [46]:
df_behavior.to_csv("../Projections/CLOCK_EPM_ROI_durations.csv",index=False)

In [47]:
df_behavior

,mouse,closed_arm_time,open_arm_time,center_time,nan_time
0,Mouse0911,241,7,40,12
1,Mouse0912,210,13,67,10
2,Mouse0913,126,64,48,62
3,Mouse0914,225,13,27,35
4,Mouse2401,78,11,92,119
5,Mouse2402,194,11,74,21
6,Mouse2403,114,47,94,45
7,Mouse30391,166,43,42,42
8,Mouse30392,220,6,42,32
9,Mouse3881,247,32,11,11
